In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"

In [2]:
from langchain_openai import ChatOpenAI
# 大语言模型
llm = ChatOpenAI(api_key="sk-f3c14e0485944adbbeb9b6fc26d930f7",
                   model="qwen-plus",
                   base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

In [ ]:
from langchain_openai import OpenAIEmbeddings
# 嵌入模型
embeddings = OpenAIEmbeddings(api_key="sk-f3c14e0485944adbbeb9b6fc26d930f7",
                              model="text-embedding-v2",
                              base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

In [3]:
from tongyi_embeddings import AliyunEmbeddings

embeddings = AliyunEmbeddings(
    api_key="sk-f3c14e0485944adbbeb9b6fc26d930f7",
    model="text-embedding-v2",  # 请根据阿里云文档确认模型名称
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"  # 请根据阿里云文档确认基础URL
)

In [4]:
from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)

预览
在这篇指南中，我们将构建一个应用，用于回答有关网站内容的疑问。我们将使用的特定网站是 Lilian Weng 的LLM 驱动式自主代理博客文章，它允许我们询问关于文章内容的疑问。

我们可以创建一个简单的索引管道和 RAG 链，用大约 50 行代码来完成这个任务。

In [5]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

Task Decomposition is the process of breaking down a complex task into smaller, more manageable steps or subtasks. This technique helps in planning and executing tasks more efficiently by simplifying them. It can be facilitated by language models through simple prompts, task-specific instructions, or with human input.
